# **Caracterización de la señal ECG**

In [76]:
# Package used for loading data from the input h5 file
from h5py import File

# Package intended to work with arrays
from numpy import array

# biosignalsnotebooks python package
import biosignalsnotebooks as bsnb

# Scientific packages
from numpy import linspace, max, min, average, std, sum, sqrt, where, argmax
from numpy import array, mean, average, linspace, where
from scipy.integrate import cumtrapz
from scipy.signal import welch
import numpy as np
import matplotlib.pyplot as plt
#from bokeh.plotting import figure, show
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Band, ColumnDataSource
from bokeh.models import BoxAnnotation

import bokeh.plotting as bp
from bokeh.io import output_notebook, show

import pywt
from bokeh.layouts import column



**Mostrar la señal de ECG**

In [79]:
# cargar la señal

relative_file_path = "C:\\Users\\Usuario\\Documents\\2024-1\\ECG_hiperventilacion.h5"
data, header = bsnb.load(relative_file_path, get_header=True)
print(data)

{'CH1': array([509, 507, 504, ..., 499, 503, 503], dtype=uint32), 'CH2': array([504, 498, 497, ..., 505, 495, 485], dtype=uint32), 'CH3': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32), 'CH4': array([529, 530, 528, ..., 510, 526, 538], dtype=uint32), 'CH5': array([39, 39, 39, ..., 39, 39, 39], dtype=uint32), 'CH6': array([8, 8, 8, ..., 8, 8, 8], dtype=uint32)}


In [81]:
#mostrar la señal
signal = data["CH2"]
fs= header["sampling rate"]
time = bsnb.generate_time(signal, fs)

# Nueva Bokeh figure
bokeh_figure = figure(x_axis_label = 'Time (s)', y_axis_label='Raw Data')
bokeh_figure.line(time, signal, legend_label="Data Original")
show(bokeh_figure)
#http://notebooks.pluxbiosignals.com/notebooks/Categories/Visualise/plot_acquired_data_single_rev.html

**Convertir de V a mV**

In [84]:
device = header["device"]
resolution = int(header["resolution"][0])
#Conversión final de unidades (a mV )
vcc = 3000 # mV
gain = 1000
#signal_mv = (((array(signal) / 2**resolution) - 0.5) * vcc) / gain

# Let's convert the signal's units, since we know it is a ECG signal
signal_mv = bsnb.raw_to_phy("ECG", "biosignalsplux", signal, resolution, "mV")
time1 = bsnb.generate_time(signal_mv, fs)

bsnb.plot([time, time1], [signal, signal_mv], legend_label=["Original Data (V)", "Original Data (mV)"], y_axis_label=["Raw Data", "Raw Data"], grid_plot=True, grid_lines=2, grid_columns=1, x_axis_label="Time (s)")

**Eliminación de la linea base**

In [87]:
#Eliminación del componente continuo de nuestra señal (cambio de línea de base mediante la resta del valor promedio)
#Esta tarea asegura una mayor estabilidad de nuestro sistema de filtrado.
# Baseline shift.
signal = array(signal_mv) - mean(signal_mv)

**Espectro de la señal de ECG original**

In [90]:
#Generación del espectro eléctrico
# Power spectrum
freq_axis_1, power_spect_1 = bsnb.plotfft(signal, fs)

bsnb.plot(freq_axis_1, power_spect_1,y_axis_label="Power Spectrum", grid_plot=True, grid_lines=1, grid_columns=1, x_axis_label="Frequency (Hz)")


**Filtro wavelet**

In [93]:
t=time

# Definir la función para la eliminación de ruido usando wavelets
def wavelet_denoise(signal, wavelet, level, threshold):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    for i in range(1, len(coeffs)):
        coeffs[i] = np.where(np.abs(coeffs[i]) < threshold * np.max(coeffs[i]), 0, coeffs[i])
    return pywt.waverec(coeffs, wavelet)
# Suponiendo que 'signal' y 't' ya están definidos
# signal = ...
# t = ...
# Parámetros para la función
wavelet = 'sym5' 
level = 4 

# Calcular los coeficientes wavelet de la señal
coeffs = pywt.wavedec(signal, wavelet, level=level)

# Calcular el umbral universal
N = len(coeffs)
sigma = np.median(np.abs(np.concatenate(coeffs[1:]))) / 0.6745
threshold = sigma * np.sqrt(2 * np.log(N)/N)
print(np.concatenate(coeffs[1:]))
# Filtrar la señal
filtered_signal = wavelet_denoise(signal, wavelet, level, threshold)

# Configuración para mostrar las gráficas en el notebook
output_notebook()

# Crear la gráfica de la señal original
p1 = figure(title="Señal ECG en Hiperventilacion", x_axis_label='Tiempo (s)', y_axis_label='Amplitud', plot_width=800, plot_height=400)
p1.line(t, signal, legend_label='Hiperventilacion', line_width=2)
p1.legend.location = "top_left"

# Crear la gráfica de la señal filtrada
p2 = figure(title="Señal ECG Filtrada", x_axis_label='Tiempo (s)', y_axis_label='Amplitud', plot_width=800, plot_height=400)
p2.line(t[:len(filtered_signal)], filtered_signal, legend_label='Señal Filtrada 2', line_width=2, color='orange')
p2.legend.location = "top_left"

# Mostrar las gráficas en una disposición de columna
show(column(p1, p2))


[-0.91715136 -0.11565755 -0.164594   ...  0.50858145 -0.43696368
 -0.15124814]


Loading BokehJS ...

**Espectro de la señal de ECG filtrada**

In [96]:
#Generación del espectro eléctrico
# Power spectrum
freq_axis_1, power_spect_1 = bsnb.plotfft(filtered_signal, fs)

bsnb.plot(freq_axis_1, power_spect_1,y_axis_label="Power Spectrum", grid_plot=True, grid_lines=1, grid_columns=1, x_axis_label="Frequency (Hz)")


**Envolvente de Energía de Shannon (SEE)**

In [54]:
# Configuración para mostrar las gráficas en el notebook
output_notebook()

# Definir la función para calcular la Envolvente de Energía de Shannon (SEE)
def shannon_energy_envelope(signal):
    # Diferenciación de primer orden
    diff_signal = np.diff(signal)
    
    # Normalización
    norm_signal = diff_signal / np.max(np.abs(diff_signal))
    
    # Transformación en señal unipolar usando la función de energía de Shannon
    shannon_energy = norm_signal**2 * (np.log(norm_signal**2))**2
    
    # Suavizado usando un filtro de media móvil
    window_width = 33
    smooth_see = np.convolve(shannon_energy, np.ones(window_width)/window_width, mode='same')
    
    return smooth_see

# Suponiendo que 'filtered_signal' ya está definido y corresponde a la señal filtrada de la Parte 1

# Calcular la Envolvente de Energía de Shannon (SEE)
see = shannon_energy_envelope(filtered_signal)

# Crear la gráfica de la señal filtrada
p1 = figure(title="Señal ECG Filtrada", x_axis_label='Tiempo (s)', y_axis_label='Amplitud', plot_width=800, plot_height=400)
p1.line(t[:len(filtered_signal)], filtered_signal, legend_label='Filtrada', line_width=2, color='orange')
p1.legend.location = "top_left"

# Crear la gráfica de la Envolvente de Energía de Shannon (SEE)
p2 = figure(title="Envolvente de Energía de Shannon (SEE)", x_axis_label='Tiempo (s)', y_axis_label='Energía', plot_width=800, plot_height=400)
p2.line(t[:len(see)], see, legend_label='SEE', line_width=2, color='green')
p2.legend.location = "top_left"

# Mostrar las gráficas en una disposición de columna
show(column(p1, p2))

Loading BokehJS ...

**Peak Energy Envelope (PEE)**

In [24]:
# Configuración para mostrar las gráficas en el notebook
output_notebook()

# Definir la función para calcular la Envolvente de Energía de Pico (PEE)
def peak_energy_envelope(signal):
    # Diferenciación de primer orden
    diff_signal = np.diff(signal)
    
    # Normalización
    norm_signal = diff_signal / np.max(np.abs(diff_signal))
    
    # Transformación en señal unipolar mediante operación de cuadrado
    peak_energy = norm_signal**2
    
    # Suavizado usando un filtro de media móvil
    window_width = 43
    smooth_pee = np.convolve(peak_energy, np.ones(window_width)/window_width, mode='same')
    
    return smooth_pee


# Calcular la Envolvente de Energía de Pico (PEE)
pee = peak_energy_envelope(see)

# Crear la gráfica de la señal filtrada
p1 = figure(title="Señal ECG Filtrada", x_axis_label='Tiempo (s)', y_axis_label='Amplitud', plot_width=800, plot_height=400)
p1.line(t[:len(filtered_signal)], filtered_signal, legend_label='Filtrada', line_width=2, color='orange')
p1.legend.location = "top_left"

# Crear la gráfica de la Envolvente de Energía de Pico (PEE)
p2 = figure(title="Envolvente de Energía de Pico (PEE)", x_axis_label='Tiempo (s)', y_axis_label='Energía', plot_width=800, plot_height=400)
p2.line(t[:len(pee)], pee, legend_label='PEE', line_width=2, color='red')
p2.legend.location = "top_left"

# Mostrar las gráficas en una disposición de columna
show(column(p1, p2))



Loading BokehJS ...

**Detectar picos R**

In [26]:
import numpy as np
import scipy.signal as sp_signal
from scipy.ndimage import uniform_filter1d
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import output_notebook

# Enable Bokeh to show plots in the notebook
output_notebook()

def find_peaks(sig):
    peaks, _ = sp_signal.find_peaks(sig)
    return peaks

def adjust_peak_positions(ecg_signal, estimated_peaks, window_size=25):
    adjusted_peaks = []
    for peak in estimated_peaks:
        start = peak - window_size if peak - window_size > 0 else 0
        end = peak + window_size if peak + window_size < len(ecg_signal) else len(ecg_signal)
        window = ecg_signal[start:end]
        true_peak = np.argmax(window) + start
        adjusted_peaks.append(true_peak)
    return adjusted_peaks

# Load example ECG data (replace with actual ECG data)
ecg_signal = filtered_signal

# 9. Detect peaks in PEE
estimated_peaks = find_peaks(pee)

# 10. Adjust peak positions to true R peak positions
r_peaks = adjust_peak_positions(ecg_signal, estimated_peaks)

# Creating Bokeh plot
p = figure(title="ECG Signal with Detected R Peaks", x_axis_label='Sample', y_axis_label='Amplitude', width=800, height=400)
p.line(range(len(ecg_signal)), ecg_signal, legend_label="ECG Signal", line_width=2)
p.circle(r_peaks, ecg_signal[r_peaks], size=10, color="red", legend_label="Detected R Peaks")

# Show plot
show(p)

Loading BokehJS ...

**Detectar picos R con codigo de biosignal**

In [28]:
import biosignalsnotebooks as bsnb
from bokeh.plotting import figure, show, output_notebook
import numpy as np

# Detectar los picos R
detected_peaks = bsnb.detect_r_peaks(filtered_signal, fs, time_units=True, plot_result=True)

# Acceder a la información necesaria para graficar
signal = detected_peaks["filtered_signal"]
peaks = detected_peaks["R-peaks"]
time = detected_peaks["time"]

# Crear una figura de Bokeh
p = figure(title="Detected R Peaks", x_axis_label='Time (s)', y_axis_label='Amplitude')

# Graficar la señal filtrada
p.line(time, signal, line_width=2, color='blue', legend_label='Filtered Signal')

# Marcar los picos R
p.circle(time[peaks], signal[peaks], size=5, color='red', legend_label='R Peaks')

# Mostrar la gráfica
output_notebook()
show(p)

TypeError: tuple indices must be integers or slices, not str

**Análisis de ECG: parámetros de variabilidad de la frecuencia cardíaca**

In [109]:
from bokeh.models import Span, Label
# Generar el tacograma utilizando bsnb
tachogram_data, tachogram_time = bsnb.tachogram(filtered_signal, fs, signal=True, out_seconds=True)
#Removal of ectopic beats
tachogram_data_NN, tachogram_time_NN = bsnb.remove_ectopy(tachogram_data, tachogram_time)
bpm_data = (1 / array(tachogram_data_NN)) * 60
#
# Maximum, Minimum and Average RR Interval
max_rr = max(tachogram_data_NN)
min_rr = min(tachogram_data_NN)
avg_rr = average(tachogram_data_NN)

# Maximum, Minimum and Average Heart Rate
max_hr = 1 / min_rr # Cycles per second
max_bpm = max_hr * 60 # BPM

min_hr = 1 / max_rr # Cycles per second
min_bpm = min_hr * 60 # BPM

avg_hr = 1 / avg_rr # Cyles per second
avg_bpm = avg_hr * 60 # BPM

# SDNN
sdnn = std(tachogram_data_NN)

time_param_dict = {"Maximum RR": max_rr, "Minimum RR": min_rr, "Average RR": avg_rr, "Maximum BPM": max_bpm, "Minimum BPM": min_bpm, "Average BPM": avg_bpm, "SDNN": sdnn}



In [111]:
# Creación de la gráfica del Tacograma
p_rr = figure(title="Tacograma con Parámetros Calculados", x_axis_label='Tiempo (s)', y_axis_label='Intervalos RR (s)', plot_width=800, plot_height=400)

# Línea del tacograma original
p_rr.line(tachogram_time_NN, tachogram_data_NN, legend_label="Tacograma Original", line_width=2, line_color="blue")

# Líneas horizontales para los valores de RR calculados
max_rr_line = Span(location=time_param_dict["Maximum RR"], dimension='width', line_color='red', line_dash='dashed', line_width=2)
min_rr_line = Span(location=time_param_dict["Minimum RR"], dimension='width', line_color='purple', line_dash='dashed', line_width=2)
avg_rr_line = Span(location=time_param_dict["Average RR"], dimension='width', line_color='orange', line_dash='dashed', line_width=2)

p_rr.add_layout(max_rr_line)
p_rr.add_layout(min_rr_line)
p_rr.add_layout(avg_rr_line)

# Marcadores para SDNN
sdnn_marker = time_param_dict["Average RR"] - time_param_dict["SDNN"]
p_rr.circle(tachogram_time_NN[np.argmin(np.abs(tachogram_data_NN - sdnn_marker))], sdnn_marker, size=10, color='brown', legend_label="SDNN")

# Etiquetas para las líneas horizontales de RR
p_rr.add_layout(Label(x=0, y=time_param_dict["Maximum RR"], text='Máximo RR', text_color='red', y_offset=5))
p_rr.add_layout(Label(x=0, y=time_param_dict["Minimum RR"], text='Mínimo RR', text_color='purple', y_offset=5))
p_rr.add_layout(Label(x=0, y=time_param_dict["Average RR"], text='Promedio RR', text_color='orange', y_offset=5))

show(p_rr)

In [113]:
# Creación de la gráfica del Heart Rate (BPM)
p_bpm = figure(title="Heart Rate (BPM) con Parámetros Calculados", x_axis_label='Tiempo (s)', y_axis_label='BPM', plot_width=800, plot_height=400)

# Línea del Heart Rate
p_bpm.line(tachogram_time_NN, bpm_data, legend_label="Heart Rate (BPM)", line_width=2, line_color="blue")

# Líneas horizontales para los valores de BPM calculados
max_bpm_line = Span(location=time_param_dict["Maximum BPM"], dimension='width', line_color='green', line_dash='dashed', line_width=2)
min_bpm_line = Span(location=time_param_dict["Minimum BPM"], dimension='width', line_color='pink', line_dash='dashed', line_width=2)
avg_bpm_line = Span(location=time_param_dict["Average BPM"], dimension='width', line_color='brown', line_dash='dashed', line_width=2)

p_bpm.add_layout(max_bpm_line)
p_bpm.add_layout(min_bpm_line)
p_bpm.add_layout(avg_bpm_line)

# Etiquetas para las líneas horizontales de BPM
p_bpm.add_layout(Label(x=0, y=time_param_dict["Maximum BPM"], text='Máximo BPM', text_color='green', y_offset=5))
p_bpm.add_layout(Label(x=0, y=time_param_dict["Minimum BPM"], text='Mínimo BPM', text_color='pink', y_offset=5))
p_bpm.add_layout(Label(x=0, y=time_param_dict["Average BPM"], text='Promedio BPM', text_color='brown', y_offset=5))

# Mostrar las gráficas

show(p_bpm)
